# <font color='056938'> Importación de librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import math
import joblib
import warnings

warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

# <font color='056938'> Datos preprocesados

In [2]:
# Extrae el ID del archivo del enlace original
file_id = '1Ca0IC51hXhq-p0ctOFllt2E1Ym1Kc3Dm'
direct_url = f'https://drive.google.com/uc?export=download&id={file_id}'

# Cargar el archivo
df = pd.read_csv(direct_url)

## <font color='157699'> Preparación y división de datos

In [3]:
X = df.drop('RENDIMIENTO_GLOBAL', axis=1)
y = df['RENDIMIENTO_GLOBAL']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print('Tamaño del conjunto de entrenamiento: ', X_train.shape)
print('Tamaño del conjunto de validación: ', X_test.shape)

Tamaño del conjunto de entrenamiento:  (554000, 50)
Tamaño del conjunto de validación:  (138500, 50)


# <font color='056938'> XGBoost Classifier

In [5]:
xgb_model = XGBClassifier(
    random_state=99,
    use_label_encoder=False,
    eval_metric='mlogloss'
)

In [6]:
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [7]:
y_pred = xgb_model.predict(X_test)

In [8]:
# Calcular accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy del modelo XGBoost: {accuracy:.4f}\n")

# Validación cruzada con 5 folds
cv_scores = cross_val_score(xgb_model, X_train, y_train, cv=5, scoring='accuracy')
print(f"Accuracy promedio (cross-validation): {cv_scores.mean():.4f}")

Accuracy del modelo XGBoost: 0.4160

Accuracy promedio (cross-validation): 0.4164


# <font color='056938'> XGBoost con optimización de hiperparámetros

In [9]:
xgb = XGBClassifier(
    random_state=42,
    use_label_encoder=False,
    eval_metric='mlogloss'
)

param_grid = {
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.1, 0.3],
    'n_estimators': [100, 200, 500],
    'gamma': [0, 0.1, 0.5]
}

grid_search_xgb = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2
)

In [10]:
grid_search_xgb.fit(X_train, y_train)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='mlogloss', feature_types=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             n_jobs=-1,
             param_grid={'gamma': [0, 0.1, 0.5],
                         'learning_rate': [0.01, 0.1, 0.3],
                         'max_depth': [3, 6, 10],
                         'n_estimators': [100, 200, 500]},
             scoring='accuracy', verbose=2)

In [11]:
best_params_xgb = grid_search_xgb.best_params_
best_score_xgb = grid_search_xgb.best_score_

print("Mejores hiperparámetros:", best_params_xgb)
print("Mejor accuracy promedio (cross-validation):", best_score_xgb)

Mejores hiperparámetros: {'gamma': 0, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 500}
Mejor accuracy promedio (cross-validation): 0.41787725631768946


In [12]:
best_xgb_model = grid_search_xgb.best_estimator_
y_pred_xgb_optimized = best_xgb_model.predict(X_test)
accuracy_xgb_optimized = accuracy_score(y_test, y_pred_xgb_optimized)

print(f"Accuracy del modelo XGBoost optimizado en el conjunto de validación: {accuracy_xgb_optimized:.4f}")

Accuracy del modelo XGBoost optimizado en el conjunto de validación: 0.4183


# <font color='056938'> Predicciones

In [29]:
file_id = '1WfDA1hut5pb_1CU8gVPX-1QA88IVAgbl'
direct_url = f'https://drive.google.com/uc?export=download&id={file_id}'

test_data = pd.read_csv(direct_url)

In [25]:
test_data = test_data.drop(columns=['PERIODO'], errors='ignore')


In [26]:
X_test = test_data.drop(columns=['ID'], errors='ignore')

y_test_pred = best_xgb_model.predict(X_test)


In [27]:
submission = pd.DataFrame({
    'ID': test_data['ID'],
    'RENDIMIENTO_GLOBAL': y_test_pred
})

# Reconvertir las clases a sus etiquetas originales (opcional si trabajamos con números codificados)
class_mapping = {0: 'bajo', 1: 'medio-bajo', 2: 'medio-alto', 3: 'alto'}
submission['RENDIMIENTO_GLOBAL'] = submission['RENDIMIENTO_GLOBAL'].map(class_mapping)

# Guardar el archivo como CSV
submission.to_csv('submission.csv', index=False)
print("Archivo de predicciones generado: submission.csv")


Archivo de predicciones generado: submission.csv


In [28]:
submission

,ID,RENDIMIENTO_GLOBAL
0,550236,medio-alto
1,98545,medio-alto
2,499179,alto
3,782980,bajo
4,785185,bajo
...,...,...
296781,496981,bajo
296782,209415,bajo
296783,239074,medio-alto
296784,963852,alto
